---

# 231003
## 목표
1. 한 타깃에 여러개 할당 시의 weight 수정

In [23]:
import os

directory = "./data"

all_items = sorted(os.listdir(directory))

In [24]:
"""
목표함수 (1)
해당 목적함수는 결정변수 theta_wt와 요격확률과 타깃의 weight의 곱의 합을 최대화함으로써 요격확률이 높을 때 표적-발사대 쌍의 발사 시점에 무기할당을 하는 의미로 해석할 수 있다.
input은, 할당여부를 나타내는 theta_wt, 요격확률 PK_wt, 타깃의 weight
"""
def objective_fun_1(theta_wt, weight, PK_wt):
    res = 0
    for t in range(T):
        
        a = 1
        for w in range(W):
            for m in range(M):
                p = 1 - PK_wt[w,t] * theta_wt[w,t,m]
                a *= p
        
        a *= weight[t]

        # print(f"타깃 {t}에 대한 성공 값 : {a}")
        res += a


    return res


In [26]:
import time
import numpy as np
import time

In [33]:
data_list = []
for data in all_items:
    path = os.path.join(directory,data)

    start_time = time.time()
    # # 데이터의 path
    # path = "data/wta_100x200x3.txt"

    # reading data from text file
    f = open(path, "r")
    lines = f.readlines()
    f.close()

    # reading first line
    # FN_T 는 타깃 당 최대 미사일 할당량
    W, T, M_W = [int(v) for v in lines[0].split()]
    # asssuming all weapons have the same availability:
    M_W = [M_W] * W

    M = sum(M_W)

    # PK_wt 배열 초기화
    PK_wt = np.zeros((W, T))

    # reading target weights, len(weight) == T
    weight=[]
    for line in lines[1:T + 1]:
        weight.append(int(line))

    # reading probabilities
    for line in lines[T + 1:]:
        splitted = line.split()
        w, t, p = int(splitted[0]), int(splitted[1]), float(splitted[2])
        PK_wt[w][t] = p

    import copy
    # 할당 여부를 나타내는 theta_wt
    theta_wt = np.zeros((W, T, M), dtype=int)

    # 발사대에 적재된 유도탄 수량 리스트 left_M_W으로 복사
    left_M_W = copy.copy(M_W)

    # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
    flattened = (PK_wt*weight).flatten()

    # 원소중 가장 큰 값 인덱스 저장
    max_index = np.argmax(flattened)

    i = 0
    m = 0

    indices_list = []

    while True :
        if i == T * W :
            break
        
        # 가장 요격확률 * 가중치가 큰 값
        v = max_index
        
        row = v // T # Weapon
        col = v % T # Target
        # print(f"Value: {flattened[v]}, Index: (Weapon : {row}, Target : {col})")
        
        # 해당 Weapon의 적재된 유도탄이 남아 있다면,
        if left_M_W[row] > 0:
        
            # 할당할 해당 무기(row)의 유도탄 개수 감소
            left_M_W[row] -= 1

            # 해당 W T M에 1 부여
            theta_wt[row][col][m] = 1

            # 무기가 할당된 indices_list에 추가
            indices_list.append(v)

            # 다음 미사일 인덱스로 이동
            m += 1

            weight[col] *= (1-PK_wt[row][col])

            # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
            flattened = (PK_wt*weight).flatten()

            max_index = np.argmax(flattened)

        
        else :
            # 미사일 소진 시 요격확률 0으로 수정
            # print(f"not enough Missile on Weapon {row}")
            PK_wt[row] = np.zeros_like(col)
            
        
        # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
        if sum(left_M_W) == 0:
            break
        
        # 다음으로 요격확률이 높은 인덱스로 이동
        # 행렬의 원소들을 1차원 배열로 변환, 이때, 타깃 가중치 추가
        flattened = (PK_wt*weight).flatten()

        max_index = np.argmax(flattened)
        # 남은 미사일 개수 출력
        # print(sum(left_M_W))

    ov = objective_fun_1(theta_wt,weight,PK_wt)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"data : {data} | objective vlaue : {ov:.4f} | Elapsed time: {elapsed_time:.2f} seconds")
    data_list.append([data,ov,elapsed_time])

data : wta_100x200x1.txt | objective vlaue : 2926.7788781577947 | Elapsed time: 1.054508924484253 seconds
data : wta_100x200x2.txt | objective vlaue : 127.80511881425672 | Elapsed time: 2.082176685333252 seconds
data : wta_100x200x3.txt | objective vlaue : 17.024566697258663 | Elapsed time: 3.0752477645874023 seconds
data : wta_150x300x1.txt | objective vlaue : 3759.0584586823175 | Elapsed time: 3.374326229095459 seconds
data : wta_150x300x2.txt | objective vlaue : 121.44052248186682 | Elapsed time: 6.963499307632446 seconds
data : wta_150x300x3.txt | objective vlaue : 13.068544459183565 | Elapsed time: 10.68991994857788 seconds
data : wta_200x400x1.txt | objective vlaue : 5042.485588724209 | Elapsed time: 7.96612548828125 seconds
data : wta_200x400x2.txt | objective vlaue : 141.28107347736008 | Elapsed time: 16.15864372253418 seconds
data : wta_200x400x3.txt | objective vlaue : 12.066955000365505 | Elapsed time: 24.681549549102783 seconds
data : wta_250x500x1.txt | objective vlaue : 6

In [34]:
#CSV 파일로 저장
import csv

filename = 'GRDsol.csv'
header = ['Data','Object Value','Runtime']

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data_list)

In [22]:
# theta_wt에서 3번째 축을 따라 합산
sol = np.sum(theta_wt, axis=2)

# 0이 아닌 원소의 인덱스 찾기
non_zero_indices = np.argwhere(sol != 0)

# 인덱스 출력
for index in non_zero_indices:
    temp_w, temp_t = tuple(index)
    print(f"x({temp_w},{temp_t})={sol[temp_w][temp_t]}")

x(0,44)=1
x(0,81)=1
x(0,117)=1
x(1,94)=1
x(1,191)=1
x(1,198)=1
x(2,47)=1
x(2,61)=1
x(2,180)=1
x(3,1)=1
x(3,7)=1
x(3,196)=1
x(4,6)=1
x(4,92)=1
x(4,187)=1
x(5,12)=1
x(5,72)=1
x(5,98)=1
x(6,88)=1
x(6,100)=1
x(6,160)=1
x(7,30)=1
x(7,89)=1
x(7,183)=1
x(8,8)=1
x(8,120)=1
x(8,166)=1
x(9,31)=1
x(9,69)=1
x(9,84)=1
x(10,26)=1
x(10,33)=1
x(10,84)=1
x(11,59)=1
x(11,60)=1
x(11,142)=1
x(12,44)=1
x(12,94)=1
x(12,144)=1
x(13,19)=1
x(13,112)=1
x(13,137)=1
x(14,54)=1
x(14,116)=1
x(14,139)=1
x(15,78)=1
x(15,99)=1
x(15,135)=1
x(16,81)=1
x(16,82)=1
x(16,186)=1
x(17,75)=1
x(17,82)=1
x(17,83)=1
x(18,2)=2
x(18,36)=1
x(19,33)=1
x(19,71)=1
x(19,109)=1
x(20,106)=1
x(20,114)=1
x(20,140)=1
x(21,110)=1
x(21,141)=2
x(22,57)=1
x(22,71)=1
x(22,184)=1
x(23,75)=1
x(23,168)=1
x(23,179)=1
x(24,29)=1
x(24,107)=1
x(24,108)=1
x(25,122)=1
x(25,161)=1
x(25,187)=1
x(26,117)=1
x(26,160)=1
x(26,185)=1
x(27,157)=1
x(27,161)=1
x(27,188)=1
x(28,11)=1
x(28,130)=1
x(28,134)=1
x(29,56)=1
x(29,105)=1
x(29,198)=1
x(30,111)=1
x(30,145)=1
